<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/Miniproyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/vision-artificial.zip' >> /dev/null  # ACTUALIZAR: ruta al fichero comprimido


Mounted at /content/gdrive


In [13]:
from sklearn.preprocessing import LabelEncoder
import os
import pandas as pd

# Especificar las rutas al directorio con las imágenes y al fichero con las etiquetas
imgtrain_dir = "/content/train/images/"
imgtrainmask_dir = "/content/train/masks/"
csvtrain_file = "/content/train.csv"

imgtest_dir = "/content/train.csv"
csvtest_file = "/content/test.csv"
# Leer el fichero CSV con las etiquetas
df = pd.read_csv(csvtrain_file, dtype={"class": "category"})
dftest = pd.read_csv(csvtest_file, dtype={"class": "category"})

# Codificar las etiquetas utilizando LabelEncoder
label_encoder = LabelEncoder()
df['class_encoded'] = label_encoder.fit_transform(df['class'])

# Convertir las etiquetas codificadas en un vector one-hot
class_one_hot = pd.get_dummies(df['class_encoded'], prefix='class')

# Renombrar las columnas del vector one-hot
class_one_hot.columns = ['normal', 'benign', 'malignant']

# Concatenar el DataFrame original con las etiquetas one-hot y la columna 'mask_filename'
df = pd.concat([df[['image_filename', 'mask_filename']], class_one_hot], axis=1)
df


,image_filename,mask_filename,normal,benign,malignant
0,image_003.png,mask_003.png,True,False,False
1,image_004.png,mask_004.png,True,False,False
2,image_008.png,mask_008.png,True,False,False
3,image_012.png,mask_012.png,True,False,False
4,image_016.png,mask_016.png,True,False,False
...,...,...,...,...,...
332,image_399.png,mask_399.png,False,False,True
333,image_400.png,mask_400.png,False,False,True
334,image_416.png,mask_416.png,False,False,True
335,image_419.png,mask_419.png,False,False,True


In [14]:
# Dividir el conjunto en entrenamiento, validación y test (70:15:15)
val_size = int(len(df) * 0.15)
test_size = int(len(df) * 0.15)

df = df.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dftest = df[:test_size]
dfval = df[test_size:test_size+val_size]
dftrain = df[test_size+val_size:]

print(f'Número de ejemplos del conjunto de entrenamiento: {dftrain.shape[0]}')
print(f'Número de ejemplos del conjunto de validación: {dfval.shape[0]}')
print(f'Número de ejemplos del conjunto de test: {dftest.shape[0]}')
dftrain = dftrain.reset_index(drop=True)
dfval = dfval.reset_index(drop=True)
dftest = dftest.reset_index(drop=True)

Número de ejemplos del conjunto de entrenamiento: 237
Número de ejemplos del conjunto de validación: 50
Número de ejemplos del conjunto de test: 50


In [15]:
import os
from PIL import Image

archivos = os.listdir(imgtrain_dir)
anchuras = []
alturas = []
for archivo in archivos:
      imagen = Image.open(os.path.join(imgtrain_dir, archivo))
      ancho, alto = imagen.size
      anchuras.append(ancho)
      alturas.append(alto)
media_anchura = sum(anchuras) / len(anchuras)
media_altura = sum(alturas) / len(alturas)

print("Media de anchura:", media_anchura)
print("Media de altura:", media_altura)

Media de anchura: 634.9139465875371
Media de altura: 514.9317507418398


In [16]:
import tensorflow as tf
from tensorflow.data import Dataset

# Dimensiones deseadas de la imagen
img_width, img_height = 224, 224
n_channels = 3                # número de canales (RGB)
n_classes = 3                 # número de clases
x_col = 'image_filename'      # nombres de las columnas en el fichero CSV
y_col = ['normal', 'benign', 'malignant']  # lista de nombres de las columnas de las etiquetas

def load_and_preprocess_image(image_filename, mask_filename, label_one_hot):
    # Cargar la imagen
    image_path = tf.strings.join([imgtrain_dir, image_filename])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=n_channels)
    image = tf.image.resize_with_pad(image, img_width, img_height)
    image = image / 255.0

    # Generar la ruta de la máscara dinámicamente
    mask_path = tf.strings.join([imgtrainmask_dir, mask_filename])

    # Cargar y procesar la máscara binaria
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)  # La máscara es una imagen en escala de grises (un solo canal)
    mask = tf.image.resize_with_pad(mask, img_width, img_height)
    mask = mask / 255.0  # Normalizar la máscara binaria (opcional)

    label = label_one_hot
    return image, mask, label


def get_dataset(df):
    image_filenames = df['image_filename'].values
    mask_filenames = df['mask_filename'].values
    labels = df[y_col].values
    dataset = tf.data.Dataset.from_tensor_slices((image_filenames, mask_filenames, labels))
    dataset = dataset.map(load_and_preprocess_image)  # Ajusta la función de carga para manejar la máscara
    return dataset

# Crear los conjuntos de datos y preparar los lotes
batch_size = 32
train_dataset = get_dataset(dftrain).batch(batch_size)
val_dataset = get_dataset(dfval).batch(batch_size)
test_dataset = get_dataset(dftest).batch(batch_size)

print(f'Número de lotes del conjunto de entrenamiento: {len(train_dataset)}')
print(f'Número de lotes del conjunto de validación: {len(val_dataset)}')
print(f'Número de lotes del conjunto de test: {len(test_dataset)}')



Número de lotes del conjunto de entrenamiento: 8
Número de lotes del conjunto de validación: 2
Número de lotes del conjunto de test: 2


In [17]:
# Crear un iterador sobre el dataset
dataset_iterator = iter(train_dataset)

# Obtener el primer elemento del dataset
sample_image, sample_mask, sample_label = next(dataset_iterator)

# Imprimir la forma y el tipo de los datos
print("Imagen:")
print(sample_image.shape)
print("Máscara:")
print(sample_mask.shape)
print("Etiqueta:")
print(sample_label)


Imagen:
(32, 224, 224, 3)
Máscara:
(32, 224, 224, 1)
Etiqueta:
tf.Tensor(
[[False  True False]
 [ True False False]
 [ True False False]
 [ True False False]
 [ True False False]
 [False  True False]
 [False  True False]
 [False False  True]
 [ True False False]
 [False  True False]
 [False  True False]
 [False False  True]
 [False False  True]
 [ True False False]
 [False  True False]
 [False False  True]
 [False  True False]
 [ True False False]
 [ True False False]
 [False  True False]
 [ True False False]
 [False  True False]
 [False  True False]
 [False  True False]
 [False False  True]
 [ True False False]
 [False  True False]
 [ True False False]
 [ True False False]
 [False False  True]
 [ True False False]
 [ True False False]], shape=(32, 3), dtype=bool)


In [18]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
import tensorflow as tf

def get_very_simple_alexnet_model(input_shape, num_classes):
    input_img = Input(shape=input_shape)
    input_mask = Input(shape=input_shape[:-1] + (1,))  # Añadir un canal adicional para la máscara binaria

    # Concatenar la imagen con la máscara binaria a lo largo del canal
    concatenated_input = Concatenate(axis=-1)([input_img, input_mask])

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(concatenated_input)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)

    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[input_img, input_mask], outputs=output)
    return model

# Definir el tamaño de entrada y el número de clases
input_shape = (img_width, img_height, 3)
n_classes = 3

# Crear el modelo
very_simple_alexnet_model = get_very_simple_alexnet_model(input_shape, n_classes)

# Definir la métrica F1
def f1_metric(y_true, y_pred):
    y_true_int = tf.argmax(y_true, axis=1)
    y_pred_int = tf.argmax(y_pred, axis=1)
    y_true_int = tf.cast(y_true_int, tf.float32)
    y_pred_int = tf.cast(y_pred_int, tf.float32)

    true_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true_int, 1), tf.equal(y_pred_int, 1)), tf.float32))
    false_positives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true_int, 0), tf.equal(y_pred_int, 1)), tf.float32))
    false_negatives = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true_int, 1), tf.equal(y_pred_int, 0)), tf.float32))

    precision = true_positives / (true_positives + false_positives + tf.keras.backend.epsilon())
    recall = true_positives / (true_positives + false_negatives + tf.keras.backend.epsilon())
    f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())

    return f1

# Compilar el modelo
very_simple_alexnet_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', f1_metric])

# Imprimir el resumen del modelo
very_simple_alexnet_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 224, 224, 1)]        0         []                            
                                                                                                  
 concatenate (Concatenate)   (None, 224, 224, 4)          0         ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 224, 224, 32)         1184      ['concatenate[0][0]']     

In [20]:
epochs = 6
history = very_simple_alexnet_model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)


TypeError: '_BatchDataset' object is not subscriptable

In [ ]:
test_loss, test_accuracy, test_f1 = very_simple_alexnet_model.evaluate(test_dataset, verbose=2)
print("test_loss: %.4f, test_acc: %.4f, test_f1: %.4f" % (test_loss, test_accuracy, test_f1))

2/2 - 3s - loss: 0.8258 - accuracy: 0.6000 - f1_metric: 0.6667 - 3s/epoch - 2s/step
test_loss: 0.8258, test_acc: 0.6000, test_f1: 0.6667


In [ ]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input


# Directorio donde se encuentran las imágenes de prueba
test_images_dir = 'test/images/'

# Obtener la lista de nombres de archivos de las imágenes de prueba
test_filenames = os.listdir(test_images_dir)

# Crear un DataFrame para almacenar las predicciones
predictions_df = pd.DataFrame({'image_filename': test_filenames})

# Crear columnas para 'normal', 'benign' y 'malignant' con valores iniciales de 0
predictions_df['normal'] = 0
predictions_df['benign'] = 0
predictions_df['malignant'] = 0

# Iterar sobre cada imagen de prueba
for filename in test_filenames:
    # Cargar la imagen y preprocesarla
    img_path = os.path.join(test_images_dir, filename)
    img = load_img(img_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión adicional para el lote

    # Realizar la predicción
    prediction = very_simple_alexnet_model.predict(img_array)
    predicted_class = np.argmax(prediction)  # Obtener la clase predicha

    # Actualizar las columnas correspondientes según la predicción
    if predicted_class == 0:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'normal'] = 1
    elif predicted_class == 1:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'benign'] = 1
    elif predicted_class == 2:
        predictions_df.loc[predictions_df['image_filename'] == filename, 'malignant'] = 1

# Ordenar el DataFrame por el nombre del archivo de imagen
predictions_df = predictions_df.sort_values(by='image_filename')

# Guardar el DataFrame en un archivo CSV
predictions_df.to_csv('test_predictions.csv', index=False)

# Imprimir las primeras filas del DataFrame para verificar
print(predictions_df.head())


1/1 [==============================] - 0s 53ms/step
   image_filename  normal  benign  malignant
54  image_005.png       1       0          0
13  image_006.png       1       0          0
23  image_007.png       1       0          0
51  image_009.png       1       0          0
8   image_010.png       1       0          0
